In [ ]:
"""
Created on Thu Jan 16 14:55 2023

Trying to do overview plots like Pierre

Author: @claraburgard
"""

In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [4]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'
clara_path='/data/cburgard/CASIMIR_SIMU/interim/CDO_PROCESSED/'

In [5]:
T_grid_vars_closed_mean10 = xr.open_dataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
T_grid_vars_open_mean10 = xr.open_dataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
T_grid_vars_open_mean10 = T_grid_vars_open_mean10.where(T_grid_vars_open_mean10['so'] > 0)

In [13]:
cellarea_closed = T_grid_vars_closed_mean10['area']
cellarea_open = T_grid_vars_open_mean10['area']

In [6]:
var_list = ['global_temp','wed_gyre','ross_gyre','ACC',
            'FRIS_melt','Ross_melt','LarsenC_melt','total_melt',
            'mar_sie_arc','sep_sie_arc','feb_sie_ant','sep_sie_ant',
            'salbot_WW','salbot_EW','salbot_WR','salbot_ER',
            'theta_WW','theta_EW','theta_WR','theta_ER']

In [7]:
var_obs_mean = xr.DataArray(data=np.array([15.0, 56.0, 20.0, 136.7, 
               155.4, 47.7, 20.7, 1500,
               15.3, 6.3, 3.3, 19.8, 
               34.9, np.nan, 35.0, np.nan, 
               np.nan, -1.95, -1.9, -1.7]), dims='var').assign_coords({'var': var_list})

In [8]:
var_obs_std = xr.DataArray(data=np.array([0, 8.0, 5.0, 7.8, 
              45.0, 34.0, 67.0, 237,
              1.0, 1.0, 0.5, 0.5, 
              0.0, np.nan, 0.0, np.nan,
              np.nan, 0.2, 0.4, 0.4]), dims='var').assign_coords({'var': var_list})

In [9]:
var_to_plot = xr.Dataset()

### GLOBAL SST
temp_open = xr.open_dataset(clara_path+'n42openc/n42openc_01-100_1Y_thetao_fldmean.nc').isel(deptht=0).squeeze()
temp_closed = xr.open_dataset(clara_path+'n42tm21/n42tm21_01-100_1Y_thetao_fldmean.nc').isel(deptht=0).squeeze()
var_to_plot['global_temp'] = xr.concat([temp_open['thetao'].assign_coords({'cav':'open'}),temp_closed['thetao'].assign_coords({'cav':'closed'})], dim='cav')

### ACC
u_open = xr.open_dataset(clara_path+'n42openc/n42openc_0-100_1Y_uo_vertsum.nc').squeeze()
u_closed = xr.open_dataset(clara_path+'n42tm21/n42tm21_0-100_1Y_uo_vertsum.nc').squeeze()
var_to_plot['ACC'] = xr.concat([u_open['uocetr_eff'].sel(x=220,y=range(79,107)).sum('y').assign_coords({'cav':'open'}),
                                u_closed['uocetr_eff'].sel(x=220,y=range(79,107)).sum('y').assign_coords({'cav':'closed'})], dim='cav')

In [10]:
### Iceshelf melt
#T_grid_vars_closed = xr.open_dataset(clara_path + 'n42tm21_00910101_01001231_1Y_grid_T.nc')
T_grid_vars_open = xr.open_dataset(clara_path + 'n42openc/n42openc_0-100_1Y_grid_isfvars.nc')
#T_grid_vars_open = T_grid_vars_open.where(T_grid_vars_open['so'] > 0)

lon = T_grid_vars_open.nav_lon
lat = T_grid_vars_open.nav_lat

mask_FRIS = (lon <= -30) & (lon >= -80.) & (lat >= -84.) & (lat <= -74.)
mask_Ross = ((lon <= -120) | (lon >= 150)) & (lat >= -87.) & (lat <= -65.)
mask_LarsenC = (lon <= -58) & (lon >= -66) & (lat >= -71.) & (lat <= -64.)

In [11]:
melt_all = (T_grid_vars_open['iceshelf_cav'] * T_grid_vars_open_mean10['area'].max('deptht') * yearinsec *10**(-12))
FRIS_melt= melt_all.where(mask_FRIS).sum(['x','y'])
Ross_melt = melt_all.where(mask_Ross).sum(['x','y'])
LarsenC_melt = melt_all.where(mask_LarsenC).sum(['x','y'])

var_to_plot['FRIS_melt'] = xr.concat([FRIS_melt.assign_coords({'cav':'open'}), FRIS_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')
var_to_plot['Ross_melt'] = xr.concat([Ross_melt.assign_coords({'cav':'open'}), Ross_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')
var_to_plot['LarsenC_melt'] = xr.concat([LarsenC_melt.assign_coords({'cav':'open'}), LarsenC_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')

var_to_plot['total_melt'] = var_to_plot['FRIS_melt'] + var_to_plot['Ross_melt'] + var_to_plot['LarsenC_melt'] + (T_grid_vars_open['iceshelf'] * T_grid_vars_open_mean10['area'].isel(deptht=0) * yearinsec * 10**(-12)).sum(['x','y'])


In [12]:
### Sea ice
si_Ant_open = xr.open_dataset(clara_path+'n42openc/n42openc_01-100_1M_icemod_Ant_SIEmask.nc')
si_Ant_closed = xr.open_dataset(clara_path+'n42tm21/n42tm21_01-100_1M_icemod_Ant_SIEmask.nc')

si_Arc_open = xr.open_dataset(clara_path+'n42openc/n42openc_01-100_1M_icemod_Arc_SIEmask.nc')
si_Arc_closed = xr.open_dataset(clara_path+'n42tm21/n42tm21_01-100_1M_icemod_Arc_SIEmask.nc')


In [43]:
cellarea_Ant_closed = cellarea_closed.sel(x=si_Ant_closed.x, y=si_Ant_closed.y)
cellarea_Arc_closed = cellarea_closed.sel(x=si_Arc_closed.x, y=si_Arc_closed.y)
cellarea_Ant_open = cellarea_open.sel(x=si_Ant_open.x, y=si_Ant_open.y)
cellarea_Arc_open = cellarea_open.sel(x=si_Arc_open.x, y=si_Arc_open.y)


In [ ]:
sie_Ant_open = (si_Ant_open['siconc'] * cellarea_Ant_open).sum(['x','y'])
sie_Ant_closed = (si_Ant_closed['siconc'] * cellarea_Ant_closed).sum(['x','y'])
sie_Arc_open = (si_Arc_open['siconc'] * cellarea_Arc_open).sum(['x','y'])
sie_Arc_closed = (si_Arc_closed['siconc'] * cellarea_Arc_closed).sum(['x','y'])


In [20]:
var_to_plot['mar_sie_arc'] = xr.concat([sie_Arc_open.where(sie_Arc_open['time_counter.month'] == 3, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Arc_closed.where(sie_Arc_closed['time_counter.month'] == 3, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav')
var_to_plot['sep_sie_ant'] = xr.concat([sie_Ant_open.where(sie_Ant_open['time_counter.month'] == 9, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Ant_closed.where(sie_Ant_closed['time_counter.month'] == 9, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav')
var_to_plot['sep_sie_arc'] = xr.concat([sie_Arc_open.where(sie_Arc_open['time_counter.month'] == 9, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Arc_closed.where(sie_Arc_closed['time_counter.month'] == 9, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav')
var_to_plot['feb_sie_ant'] = xr.concat([sie_Ant_open.where(sie_Ant_open['time_counter.month'] == 2, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Ant_closed.where(sie_Ant_closed['time_counter.month'] == 2, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav')


    
#'mar_sie_arc','sep_sie_arc','feb_sie_ant','sep_sie_ant'

In [27]:
si_Arc_closed['cell_area'].plot()

In [ ]:
sie_Arc_open['siconc'].where(sie_Arc_open['time_counter.month'] == 3, drop=True).squeeze()

In [ ]:
var_to_plot['total_melt']

In [ ]:
SIE_Arc_03_open = sie_Arc_open['siconc'].where(sie_Arc_open['time_counter.month'] == 3, drop=True).squeeze()
SIE_Arc_03_open['time_counter'] = np.arange(len(SIE_Arc_03_open.time_counter))
SIE_Arc_03_open

In [ ]:
SIE_Arc_03_open.plot()

In [ ]:
f = plt.figure()
f.set_size_inches(8.25*5, 8.25*5)

ax={}

leg_hdl = []

i = 0


for vv in var_list:
    
    ax[i] = f.add_subplot(5,5,i+1)

    if vv in list(var_to_plot.keys()):
        ax[i].plot(var_to_plot[vv].sel(cav='open'), color='deepskyblue')
        ax[i].plot(var_to_plot[vv].sel(cav='closed'), color='royalblue')
    
    ax[i].axhline(y=var_obs_mean.sel(var=vv), color='black', linewidth=2)
    ax[i].fill_between(x=np.arange(0,100),y1=var_obs_mean.sel(var=vv)-var_obs_std.sel(var=vv), y2=var_obs_mean.sel(var=vv)+var_obs_std.sel(var=vv), color='grey',alpha=0.2)

    ax[i].set_title(vv)

    i = i+1
#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
